In [1]:
import sys
sys.path.append("..")

import numpy as np
from tqdm import tqdm
from src.datamodules.cars196 import Cars196
from src.datamodules.cub200 import Cub200
from src.datamodules.stanford_online_products import StanfordOnlineProducts
from glob import glob
from PIL import Image

/Users/konstantinkobs/code/2022-dml-rrr/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [88]:
# dataset_name = "cars196"
dataset_name = "cub200"
# dataset_name = "sop"

datasets = {
    "cars196": Cars196,
    "cub200": Cub200,
    "sop": StanfordOnlineProducts
}

dataset = datasets[dataset_name](root="../data", mode="test")

In [57]:
np.random.seed(len(dataset))

In [58]:
indices = np.arange(len(dataset))
np.random.shuffle(indices)
indices = indices[:20]

In [59]:
indices

array([11413, 45579, 27648, 16805, 30467,  5557, 12607, 49446, 30694,
       39827, 32396, 57697, 51035, 57649, 26907, 18776, 35631, 14986,
       20979, 10120])

In [60]:
for i in tqdm(indices):
    item = dataset[i]
    item["inputs"].save(f"gt_masks/{dataset_name}/{i}_image.jpeg")
    item["masks"].save(f"gt_masks/{dataset_name}/{i}_mask_pred.png")

100%|██████████| 20/20 [00:00<00:00, 56.68it/s]


# Compute metrics between ground truth and predicted masks

In [7]:
# dataset_name = "cars196"
dataset_name = "cub200"
# dataset_name = "sop"

In [8]:
def scoring(mask, mask_pred):
    # binarize input
    mask[mask < 128] = 0
    mask[mask >= 128] = 1

    # binarize prediction
    mask_pred[mask_pred < 128] = 0
    mask_pred[mask_pred >= 128] = 1

    s = {}
    
    # intersection over union
    intersection = np.logical_and(mask, mask_pred)
    union = np.logical_or(mask, mask_pred)
    s["IoU"] = np.sum(intersection) / np.sum(union)

    # overlap
    s["overlap"] = np.sum(intersection) / np.sum(mask)

    return s

In [9]:
images = sorted(glob(f"gt_masks/{dataset_name}/*_image.jpeg"))
masks_pred = sorted(glob(f"gt_masks/{dataset_name}/*_mask_pred.png"))
masks = sorted(glob(f"gt_masks/{dataset_name}/*_mask.png"))

In [10]:
scores = []

for image, mask_pred, mask in zip(images, masks_pred, masks):
    mask_pred = Image.open(mask_pred).convert("L")
    mask = Image.open(mask).convert("L")
    mask = np.array(mask)
    mask_pred = np.array(mask_pred)
    score = scoring(np.array(mask), np.array(mask_pred))
    scores.append(score)

In [11]:
for score_name in scores[0].keys():
    s = [s[score_name] for s in scores]
    print(f"{score_name}\n mean: {np.mean(s)}, std: {np.std(s)}, min: {np.min(s)}, max: {np.max(s)}, median: {np.median(s)}, q25: {np.percentile(s, 25)}, q75: {np.percentile(s, 75)}")


IoU
 mean: 0.8365447716208436, std: 0.22877790151204158, min: 0.0, max: 0.9663508858267716, median: 0.9211384771344702, q25: 0.8735145610769534, q75: 0.9447039241369721
overlap
 mean: 0.9206427562342332, std: 0.21488953374428002, min: 0.0, max: 0.999605730056512, median: 0.9790142667445486, q25: 0.9641976595811851, q75: 0.9866641434030051
